In [64]:
unique_idx_current = 0
current_idx = 0
with open(biomarker_names_file_out, 'w') as f:
    for i,biomarker in tqdm(enumerate(biomarkers)):
        if biomarker_exists[i]:  # If biomarker doesn't even exist, just skip.
            if unique_idx_current==len(no_ld_idx):
                break
                print('here')
            elif current_idx==no_ld_idx[unique_idx_current]:
                if is_abundant[unique_idx_current]: 
                    print('here')
                    f.write('\t'.join([variants[x] for x in biomarker]) + '\t' + '\t'.join([str(x) for x in biomarker]) + '\n')
                unique_idx_current += 1
            current_idx += 1
print("Writing to files...")
#scipy.sparse.save_npz(asv_vs_biomarker_file_out, asv_biomarker)  # Save sparse matrix of ASV x biomarker.
#scipy.sparse.save_npz(person_biomarker_file_out, person_biomarker)  # Save sparse matrix of ASV x biomarker.

1596it [00:00, 506094.29it/s]

Writing to files...


In [71]:
unique_idx_current

527

In [5]:
import scipy.sparse
import numpy as np
import pandas as pd

In [47]:
def sparse_unique_columns(M):
    M = M.tocsc()
    m, n = M.shape
    if not M.has_sorted_indices:
        M.sort_indices()
    if not M.has_canonical_format:
        M.sum_duplicates()
    sizes = np.diff(M.indptr)
    idx = np.argsort(sizes)
    Ms = M@scipy.sparse.csc_matrix((np.ones((n,)), idx, np.arange(n+1)), (n, n))
    ssizes = np.diff(Ms.indptr)
    ssizes[1:] -= ssizes[:-1]
    grpidx, = np.where(ssizes)
    grpidx = np.concatenate([grpidx, [n]])
    if ssizes[0] == 0:
        counts = [np.array([0, grpidx[0]])]
    else:
        counts = [np.zeros((1,), int)]
    ssizes = ssizes[grpidx[:-1]].cumsum()
    for i, ss in enumerate(ssizes):
        gil, gir = grpidx[i:i+2]
        pl, pr = Ms.indptr[[gil, gir]]
        dv = Ms.data[pl:pr].view(f'V{ss*Ms.data.dtype.itemsize}')
        iv = Ms.indices[pl:pr].view(f'V{ss*Ms.indices.dtype.itemsize}')
        idxi = np.lexsort((dv, iv))
        dv = dv[idxi]
        iv = iv[idxi]
        chng, = np.where(np.concatenate(
            [[True], (dv[1:] != dv[:-1]) | (iv[1:] != iv[:-1]), [True]]))
        counts.append(np.diff(chng))
        idx[gil:gir] = idx[gil:gir][idxi]
    counts = np.concatenate(counts)
    nu = counts.size - 1
    uniques = M@scipy.sparse.csc_matrix((np.ones((nu,)), idx[counts[:-1].cumsum()],
                                   np.arange(nu + 1)), (n, nu))
    return uniques, idx[counts[:-1].cumsum()], counts[1:]

In [48]:
uniques, idx = np.unique(pd.DataFrame(asv_biomarker.todense()), return_index=True, axis=1)

In [7]:
asv_biomarker = scipy.sparse.load_npz('/home/groups/dpwall/briannac/sequence_based_biomarkers/results/generate_biomarkers/asv_vs_biomarker_sbb2_testing_autism.npz')
df = pd.DataFrame(asv_biomarker)
df

,0
0,"(0, 202)\tTrue\n (0, 208)\tTrue\n (0, 211)..."
1,"(0, 202)\tTrue\n (0, 208)\tTrue\n (0, 211)..."
2,"(0, 208)\tTrue\n (0, 211)\tTrue\n (0, 217)..."
3,"(0, 166)\tTrue\n (0, 170)\tTrue\n (0, 174)..."
4,"(0, 166)\tTrue\n (0, 174)\tTrue\n (0, 175)..."
...,...
5244,"(0, 142)\tTrue\n (0, 143)\tTrue\n (0, 147)..."
5245,"(0, 166)\tTrue\n (0, 170)\tTrue\n (0, 174)..."
5246,"(0, 166)\tTrue\n (0, 170)\tTrue\n (0, 174)..."
5247,"(0, 202)\tTrue\n (0, 208)\tTrue\n (0, 211)..."


In [ ]:
_, idx = np.unique([tuple(asv_biomarker[i,:].indices) for i in range(np.shape(asv_biomarker)[0])], return_index=True)
idx = sorted(idx)

print("Number of biomarkers after LD filtering: ", len(idx))


#np.save('%sperson_variant%d_condensed' %  (output_dir,order), person_biomarker)
biomarkers = combinations(range(len(variants)), order)

print('Computing biomarker names...')
with open(biomarker_names_file_out, 'w') as f:
    for i,biomarker in tqdm(enumerate(biomarkers)):
        if biomarker_exists[i] & (i in idx):  
            f.write('\t'.join([variants[x] for x in biomarker]) + '\t' + '\t'.join([str(x) for x in biomarker]) + '\n')
print("Writing to file")
asv_biomarker = scipy.sparse.csc_matrix(asv_biomarker[:,idx]).transpose()
person_biomarker = scipy.sparse.csc_matrix(person_biomarker[:,idx])

#with open(biomarker_names_file_out, 'w') as f:
#    f.writelines([b + '\n' for b in biomarker_names])

scipy.sparse.save_npz(asv_vs_biomarker_file_out, asv_biomarker)  # Save sparse matrix of ASV x biomarker.
scipy.sparse.save_npz(person_biomarker_file_out, person_biomarker)  # Save sparse matrix of ASV x biomarker.
            
            
            
#biomarker_names = ['\t'.join(
#    [variants[x] for x in biomarker]) + '\t' + '\t'.join([str(x) for x in biomarker]) for biomarker,biomarker_exist in tqdm(zip(biomarkers, biomarker_exists)) if biomarker_exist]

#print('removing Biomarkers in LD...')
#biomarkers = combinations(range(len(variants)), order)
#def filter(array, thresh):
#    count = Counter(array)
#    return tuple(sorted([i for i in count if count[i]>=thresh]))

#asvs_per_biomarker = [
#    filter(asv_variant[:,biomarker].indices, order) for biomarker,biomarker_exist in tqdm(zip(
#        biomarkers, biomarker_exists)) if biomarker_exist]
#_, idx = np.unique(asvs_per_biomarker, return_index=True)
#print(np.shape(idx))



#print('Computing biomarker_names...')
#biomarker_names = ['\t'.join(
#    [variants[x] for x in biomarker]) + '\t' + '\t'.join([str(x) for x in biomarker]) for biomarker,biomarker_exist in tqdm(zip(biomarkers, biomarker_exists)) if biomarker_exist] 

#print('Computing ASV_vs_biomarker...')
#asv_vs_biomarker = scipy.sparse.lil_matrix((np.shape(asv_variant)[0],len(biomarker_names)), dtype=bool)
#for biomarker,asv in tqdm(enumerate(asvs_per_biomarker)):
#    asv_vs_biomarker[asv,biomarker]= True
print("Writing to file")
asv_biomarker = scipy.sparse.csc_matrix(biomarker_asv[idx,:]).transpose()
person_biomarker = scipy.sparse.csc_matrix(person_biomarker[:,idx])

#with open(biomarker_names_file_out, 'w') as f:
#    f.writelines([b + '\n' for b in biomarker_names])

scipy.sparse.save_npz(asv_vs_biomarker_file_out, asv_biomarker)  # Save sparse matrix of ASV x biomarker.
scipy.sparse.save_npz(person_biomarker_file_out, person_biomarker)  # Save sparse matrix of ASV x biomarker.